# Agents for Impact: Medical Diet Navigator Challenge

## Step 2

This notebook configures a Google Agent Development Kit (ADK) multi-agent system with three agents.

1. The BigQuery Agent can query the USDA food database imported in Step 1. This allows information about foods and nutrition to be retrieved by the agent.

2. The Allergy Research Agent uses the Google Search tool to search the Allergen Online website (http://www.allergenonline.org/). This website is designed to help assess the safety of proteins that may be introduced into foods through genetic engineering, as novel foods, or through food processing methods.

3. The Main (root) Agent receives user requests and can utilize the first two agents as subagents to provide information when responding to users.

After the agents are configured, there is some test code. Finally, the agent application is deployed to Google Agent Engine.



In [ ]:
! pip install google-cloud-aiplatform[adk,agent_engines] google-auth google-genai httpx beautifulsoup4 --quiet --upgrade

In [ ]:
# @title Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# @title Setup Project and Vertex AI
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

# Used by the BigQuery USDA Agent
DATASET_NAME = "usda_dataset"


In [ ]:
# @title Create a Storage Bucket (needed for Agent Engine Deployment)

from google.cloud import storage
from uuid import uuid4

def create_unique_bucket(location: str = "US", prefix: str = "nbkt") -> storage.Bucket:
    """
    Create a Google Cloud Storage bucket with a randomly generated, globally unique name.

    Args:
        location: GCS bucket location/region (e.g., "US", "EU", "us-central1").
        prefix:   Optional short prefix for readability; must be lowercase letters/digits/hyphens.

    Returns:
        The created google.cloud.storage.bucket.Bucket object.
    """
    client = storage.Client()
    # Bucket name rules: 3–63 chars, lowercase letters/digits/hyphens, must start/end with letter or digit.
    # Use a UUID4 hex (32 chars) trimmed to keep name short while being vanishingly collision-prone.
    unique_suffix = uuid4().hex[:20]  # 20 hex chars = 80 bits of entropy
    bucket_name = f"{prefix}-{unique_suffix}"

    # Very unlikely collision; still, try once more if it happens.
    try:
        bucket = client.bucket(bucket_name)
        bucket = client.create_bucket(bucket, location=location)
        print(f"Created bucket: {bucket.name} (location: {bucket.location}) in project: {client.project}")
        return bucket
    except Exception as e:
        if "You already own this bucket" in str(e) or "Conflict" in str(e) or "already exists" in str(e):
            # Regenerate once and retry
            bucket_name = f"{prefix}-{uuid4().hex[:20]}"
            bucket = client.bucket(bucket_name)
            bucket = client.create_bucket(bucket, location=location)
            print(f"Created bucket on retry: {bucket.name} (location: {bucket.location}) in project: {client.project}")
            return bucket
        raise

# Example usage:
BUCKET = create_unique_bucket(location="US", prefix="agent-engine-staging")
STAGING_BUCKET = BUCKET.name

print(STAGING_BUCKET)


In [ ]:
# @title Initialize the Vertex AI SDK and set the Gemini version
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=f"gs://{STAGING_BUCKET}",
)

MODEL = "gemini-2.5-flash"

In [ ]:
# @title These ENV variables are used by Gemini Later
import os

# Tell google-genai to use Vertex AI and where
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

In [ ]:
# @title Authenticate to BQ

import google.auth
from google.cloud import bigquery
creds, proj = google.auth.default()
print("ADC project:", proj)

# This just tests to ensure we are authenticated to BQ
bq = bigquery.Client(project=PROJECT_ID, credentials=creds)
bq.query("SELECT 1").result()
print("BigQuery query OK")

In [ ]:
# @title This just tests to ensure we can authenticate with Gemini.

# Quick sanity check using google-genai directly:
from google import genai
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
resp = client.models.generate_content(model="gemini-2.5-flash", contents="hello")
print(resp.text)


In [ ]:
# @title ADK Imports
import asyncio
import os
import google.auth
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.bigquery import BigQueryCredentialsConfig, BigQueryToolset
from google.adk.tools.bigquery.config import BigQueryToolConfig, WriteMode
from google.adk.tools import google_search
from google.adk.tools import agent_tool


In [ ]:
# @title Build the BigQuery Agent. Uses the BigQuery tool to access the BQ Dataset created earlier


# Uses Application Default Credentials for BigQuery (gcloud or service account).
adc, _ = google.auth.default()
bq_credentials = BigQueryCredentialsConfig(credentials=adc)

# Read-only tool config (blocks DDL/DML). You can change to WriteMode.ALLOWED later if needed.
bq_tool_cfg = BigQueryToolConfig(write_mode=WriteMode.BLOCKED)

# Instantiate the BigQuery toolset
bq_tools = BigQueryToolset(
    credentials_config=bq_credentials,
    bigquery_tool_config=bq_tool_cfg
)

DB_SCHEMA = """

[{
  "table_name": "lab_method_nutrient",
  "fields": [{
    "column_name": "lab_method_id",
    "data_type": "INT64"
  }, {
    "column_name": "nutrient_id",
    "data_type": "INT64"
  }]
}, {
  "table_name": "food_update_log_entry",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }, {
    "column_name": "last_updated",
    "data_type": "DATE"
  }]
}, {
  "table_name": "input_food",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_of_input_food",
    "data_type": "INT64"
  }, {
    "column_name": "seq_num",
    "data_type": "STRING"
  }, {
    "column_name": "amount",
    "data_type": "STRING"
  }, {
    "column_name": "ingredient_code",
    "data_type": "STRING"
  }, {
    "column_name": "ingredient_description",
    "data_type": "STRING"
  }, {
    "column_name": "unit",
    "data_type": "STRING"
  }, {
    "column_name": "portion_code",
    "data_type": "STRING"
  }, {
    "column_name": "portion_description",
    "data_type": "STRING"
  }, {
    "column_name": "gram_weight",
    "data_type": "STRING"
  }, {
    "column_name": "retention_code",
    "data_type": "STRING"
  }]
}, {
  "table_name": "sub_sample_food",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id_of_sample_food",
    "data_type": "INT64"
  }]
}, {
  "table_name": "food",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "data_type",
    "data_type": "STRING"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }, {
    "column_name": "food_category_id",
    "data_type": "INT64"
  }, {
    "column_name": "publication_date",
    "data_type": "DATE"
  }]
}, {
  "table_name": "foundation_food",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "NDB_number",
    "data_type": "INT64"
  }, {
    "column_name": "footnote",
    "data_type": "STRING"
  }]
}, {
  "table_name": "market_acquisition",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "brand_description",
    "data_type": "STRING"
  }, {
    "column_name": "expiration_date",
    "data_type": "DATE"
  }, {
    "column_name": "label_weight",
    "data_type": "STRING"
  }, {
    "column_name": "location",
    "data_type": "STRING"
  }, {
    "column_name": "acquisition_date",
    "data_type": "DATE"
  }, {
    "column_name": "sales_type",
    "data_type": "STRING"
  }, {
    "column_name": "sample_lot_nbr",
    "data_type": "STRING"
  }, {
    "column_name": "sell_by_date",
    "data_type": "DATE"
  }, {
    "column_name": "store_city",
    "data_type": "STRING"
  }, {
    "column_name": "store_name",
    "data_type": "STRING"
  }, {
    "column_name": "store_state",
    "data_type": "STRING"
  }, {
    "column_name": "upc_code",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_protein_conversion_factor",
  "fields": [{
    "column_name": "food_nutrient_conversion_factor_id",
    "data_type": "INT64"
  }, {
    "column_name": "value",
    "data_type": "FLOAT64"
  }]
}, {
  "table_name": "lab_method_code",
  "fields": [{
    "column_name": "lab_method_id",
    "data_type": "INT64"
  }, {
    "column_name": "code",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_nutrient_conversion_factor",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }]
}, {
  "table_name": "food_calorie_conversion_factor",
  "fields": [{
    "column_name": "food_nutrient_conversion_factor_id",
    "data_type": "INT64"
  }, {
    "column_name": "protein_value",
    "data_type": "FLOAT64"
  }, {
    "column_name": "fat_value",
    "data_type": "FLOAT64"
  }, {
    "column_name": "carbohydrate_value",
    "data_type": "FLOAT64"
  }]
}, {
  "table_name": "food_portion",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "seq_num",
    "data_type": "INT64"
  }, {
    "column_name": "amount",
    "data_type": "FLOAT64"
  }, {
    "column_name": "measure_unit_id",
    "data_type": "INT64"
  }, {
    "column_name": "portion_description",
    "data_type": "STRING"
  }, {
    "column_name": "modifier",
    "data_type": "STRING"
  }, {
    "column_name": "gram_weight",
    "data_type": "FLOAT64"
  }, {
    "column_name": "data_points",
    "data_type": "INT64"
  }, {
    "column_name": "footnote",
    "data_type": "STRING"
  }, {
    "column_name": "min_year_acquired",
    "data_type": "INT64"
  }]
}, {
  "table_name": "sample_food",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }]
}, {
  "table_name": "lab_method",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }, {
    "column_name": "technique",
    "data_type": "STRING"
  }]
}, {
  "table_name": "nutrient",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }, {
    "column_name": "unit_name",
    "data_type": "STRING"
  }, {
    "column_name": "nutrient_nbr",
    "data_type": "FLOAT64"
  }, {
    "column_name": "rank",
    "data_type": "FLOAT64"
  }]
}, {
  "table_name": "food_nutrient",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "nutrient_id",
    "data_type": "INT64"
  }, {
    "column_name": "amount",
    "data_type": "FLOAT64"
  }, {
    "column_name": "data_points",
    "data_type": "INT64"
  }, {
    "column_name": "derivation_id",
    "data_type": "INT64"
  }, {
    "column_name": "min",
    "data_type": "FLOAT64"
  }, {
    "column_name": "max",
    "data_type": "FLOAT64"
  }, {
    "column_name": "median",
    "data_type": "FLOAT64"
  }, {
    "column_name": "footnote",
    "data_type": "STRING"
  }, {
    "column_name": "min_year_acquired",
    "data_type": "INT64"
  }]
}, {
  "table_name": "measure_unit",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }]
}, {
  "table_name": "acquisition_samples",
  "fields": [{
    "column_name": "fdc_id_of_sample_food",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id_of_acquisition_food",
    "data_type": "INT64"
  }]
}, {
  "table_name": "agricultural_samples",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "acquisition_date",
    "data_type": "DATE"
  }, {
    "column_name": "market_class",
    "data_type": "STRING"
  }, {
    "column_name": "treatment",
    "data_type": "STRING"
  }, {
    "column_name": "state",
    "data_type": "STRING"
  }]
}, {
  "table_name": "sub_sample_result",
  "fields": [{
    "column_name": "food_nutrient_id",
    "data_type": "INT64"
  }, {
    "column_name": "adjusted_amount",
    "data_type": "FLOAT64"
  }, {
    "column_name": "lab_method_id",
    "data_type": "INT64"
  }, {
    "column_name": "nutrient_name",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_attribute_type",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_category",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "code",
    "data_type": "INT64"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_attribute",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "seq_num",
    "data_type": "INT64"
  }, {
    "column_name": "food_attribute_type_id",
    "data_type": "INT64"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }, {
    "column_name": "value",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_component",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "STRING"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }, {
    "column_name": "pct_weight",
    "data_type": "FLOAT64"
  }, {
    "column_name": "is_refuse",
    "data_type": "BOOL"
  }, {
    "column_name": "gram_weight",
    "data_type": "FLOAT64"
  }, {
    "column_name": "data_points",
    "data_type": "INT64"
  }, {
    "column_name": "min_year_acqured",
    "data_type": "INT64"
  }]
}]
"""

# Instruct the agent to **only** use your dataset
INSTR = f"""
You are a data analysis agent with access to BigQuery tools.
The dataset you have access to contains information from the USDA about foods and nutrician information.
Only query the dataset `{PROJECT_ID}.{DATASET_NAME}`.
Fully qualify every table as `{PROJECT_ID}.{DATASET_NAME}.<table>`.
Never perform DDL/DML; SELECT-only. Return the SQL you ran along with a concise answer.
Here is the database schema, please study it {DB_SCHEMA}
"""

usda_bigquery_agent = Agent(
    model=MODEL,         # Works with ADK; requires a Gemini API key or Vertex AI setup
    name="usda_food_information_bigquery_agent",
    description="""Analyzes tables in a BigQuery dataset that contains food information from the USDA. Tables.""",
    instruction=INSTR,
    tools=[bq_tools],
)

In [ ]:
# @title Allergy Research Agent

from bs4 import BeautifulSoup
from google.adk.agents import Agent
from google.adk.tools import VertexAiSearchTool
from google.genai import types
from google.adk.tools import google_search


# ---- Agent that uses Allergy Online webite Data Store ----
INSTR = f"""
You are an allergen researcher.
Also, use your own knowledge about allergies and health.
You can also use the Google Search tool to find relevant information on the web.
When you use the Google Search tool, always cite the source of the information you find.
"""

MODEL = "gemini-2.5-flash"

agent_generation = types.GenerateContentConfig(
    temperature=0.6,
    top_p=0.9,
    max_output_tokens=32768,
)

allergen_research_agent = Agent(
    model=MODEL,   # or your preferred Gemini model
    name="allergy_research_agent",
    description=f"Answer questions about allergies and related health concerns.",
    instruction=INSTR,
    tools=[
        google_search
    ],
    generate_content_config=agent_generation,
)


In [ ]:
# @title Define the Image Agent
from __future__ import annotations

import time, uuid
from typing import Any, Dict, List, Optional

from google import genai
from google.genai import types
from google.cloud import storage
from google.adk.agents import Agent

# =========================
# Config
# =========================
IMAGE_BUCKET = "food-agent-generated-images-dar"  # must be public

# =========================
# Helpers
# =========================
def _ext_for_mime(mime_type: str) -> str:
    m = (mime_type or "").lower()
    if "jpeg" in m or "jpg" in m:
        return "jpg"
    if "webp" in m:
        return "webp"
    if "png" in m:
        return "png"
    if "gif" in m:
        return "gif"
    # default
    return "png"

def _public_gcs_url(bucket: str, blob_name: str) -> str:
    # Public URL that works without auth if bucket is public
    return f"https://storage.googleapis.com/{bucket}/{blob_name}"

def _upload_bytes_to_gcs(
    data: bytes,
    bucket: str,
    mime_type: str = "image/png",
    prefix: str = "generated",
) -> Dict[str, str]:
    client = storage.Client()
    bucket_obj = client.bucket(bucket)

    ext = _ext_for_mime(mime_type)
    blob_name = f"{prefix}/{int(time.time())}-{uuid.uuid4().hex}.{ext}"

    blob = bucket_obj.blob(blob_name)
    # Optionally set cache control if you want CDN/browser caching
    # blob.cache_control = "public, max-age=86400"
    blob.upload_from_string(data, content_type=mime_type)
    public_url = _public_gcs_url(bucket, blob_name)

    return {
        "gcs_uri": f"gs://{bucket}/{blob_name}",
        "public_url": public_url,
        "mime_type": mime_type,
        "filename": blob_name,
        "markdown": f'![{blob_name}]({public_url} "Generated by Imagen")',
    }

# =========================
# Tool
# =========================
def generate_image_tool(
    prompt: str,
    *,
    bucket: str = IMAGE_BUCKET,
    n: int = 1,
    return_text: bool = False,
) -> Dict[str, Any]:
    """
    Generate image(s) with Gemini 2.5 Flash Image (preview),
    upload to a (public) GCS bucket, and return lightweight URLs.

    Returns:
        {
          "status": "success",
          "images": [
            {"public_url": "...", "gcs_uri": "...", "mime_type": "...", "filename": "..."}
          ],
          "text": "optional textual output (if return_text=True)"
        }
    """
    #client = genai.Client(vertexai=True, api_key=None)

    client = genai.Client(vertexai=True, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location="global")


    contents = [types.Content(role="user", parts=[types.Part.from_text(text=prompt)])]
    # Keep tokens lean: default to IMAGE-only; include TEXT only if requested
    response_modalities = ["IMAGE"] + (["TEXT"] if return_text else [])

    cfg = types.GenerateContentConfig(
        temperature=1.0,
        top_p=0.95,
        max_output_tokens=4096,
        response_modalities=response_modalities,
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
    )

    try:
        stream = client.models.generate_content_stream(
            model="gemini-2.5-flash-image-preview",
            contents=contents,
            config=cfg,
        )

        outputs: List[Dict[str, str]] = []
        text_out: List[str] = []

        for chunk in stream:
            cand = getattr(chunk, "candidates", [None])[0]
            if not cand or not cand.content or not getattr(cand.content, "parts", None):
                continue

            for part in cand.content.parts:
                # Optional text (captions/notes)
                if getattr(part, "text", None):
                    text_out.append(part.text)

                # Image bytes
                inline = getattr(part, "inline_data", None)
                if inline and getattr(inline, "data", None):
                    info = _upload_bytes_to_gcs(
                        inline.data,
                        bucket=bucket,
                        mime_type=inline.mime_type or "image/png",
                    )
                    outputs.append(info)
                    if len(outputs) >= n:
                        break

            if len(outputs) >= n:
                break

        if not outputs and not text_out:
            return {"status": "error", "error_message": "No image or text returned."}

        result: Dict[str, Any] = {"status": "success", "images": outputs}
        if return_text and text_out:
            result["text"] = "\n".join(text_out)
        return result

    except Exception as e:
        print("ERROR generating image:", e)
        return {"status": "error", "error_message": str(e)}

# =========================
# Agent
# =========================
agent_generation = types.GenerateContentConfig(
    temperature=0.6,
    top_p=0.9,
    max_output_tokens=2048,
)

image_agent = Agent(
    model="gemini-2.5-flash",   # planner/brain model
    name="imagen_tool_agent",
    instruction=(
        "You generate images based on user prompts. "
        "When asked for an image, call `generate_image_tool` with a concise visual prompt. "
        "Return the Markdown from the tool output to the user so it can be displayed on their website."
    ),
    description="Agent that creates images via a custom tool powered by Gemini 2.5 Flash Image (preview).",
    tools=[generate_image_tool],
    generate_content_config=agent_generation,
)


In [ ]:
# @title Define the Main (root) Agent

MODEL = "gemini-2.5-flash"

agent_generation = types.GenerateContentConfig(
    temperature=0.6,
    top_p=0.9,
    max_output_tokens=32768,
)

MAIN_AGENT_INSTRUCTIONS="""
You are a friendly food and nutrician agent.
Answer questions related to food, nutrician, allergies, dietary health, and other inquiries related to thise things.
You have 2 helper agents.
- The usda_bigquery_agent has access to a large database from the USDA containing all sorts to food-related information.
- The image_agent generate images if requested
You can use your tool to search for information about allergies and related health concerns online.
When you use the Google Search tool, always cite the source of the information you find.
"""

root_agent = Agent(
    name="main_agent",
    model=MODEL,
    description="Provides Answers to Users Food and Allergy Questions.",
    instruction=MAIN_AGENT_INSTRUCTIONS,
    tools=[agent_tool.AgentTool(agent=allergen_research_agent)],
    sub_agents=[usda_bigquery_agent, image_agent],
    generate_content_config=agent_generation,
)

In [ ]:
# @title Create the Runner that uses our root agent

# --- Session Management ---
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "usda_food_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=root_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

In [ ]:
# @title Define Agent Interaction Function


async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  return final_response_text

In [ ]:
# @title Run the Initial Conversation

questions = [
     "List all tables in the dataset.",
    "Top 10 foods by protein per 100g from the table `food_nutrient` joined with `nutrient`.",
    "How many rows are in `food`?",
    "Give me a dinner recommendation for someone with allergies to dairy and nuts. Include a meat, starch, and vegetable"
    "Create an Image of a prepared steak dinner in a cozy atmosphere"
    ]

# We need an async function to await our interaction helper
async def run_conversation():
  for q in questions:
    response = await call_agent_async(query=q,runner=runner,user_id=USER_ID,session_id=SESSION_ID)
    print("-----------------------------")
    print(f"User: {q}")
    print(f"Model: {response}")
    print("-----------------------------")


# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()

# --- OR ---

# Uncomment the following lines if running as a standard Python script (.py file):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"An error occurred: {e}")

In [ ]:
# @title Deploy to Agent Engine
from vertexai import agent_engines

# Wrap the agent in an AdkApp object
app = agent_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

remote_app = agent_engines.create(
    display_name = "usda-food-agent",
    agent_engine=app,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]",
        "google-auth",
        "google-genai",
        "httpx",
        "beautifulsoup4",
        "google-cloud-storage",
        "google-cloud-bigquery",
        "google-cloud-logging",
        "google-cloud-resource-manager",
        "google-genai"
    ]
)

print(f"Deployment finished!")
print(f"Resource Name: {remote_app.resource_name}")
# Resource Name: "projects/{PROJECT_NUMBER}/locations/{LOCATION}/reasoningEngines/{RESOURCE_ID}"